In [2]:
import pandas as pd
import re

profile_df = pd.read_csv('../static/data/original/neighbourhood-profiles-2016-csv.csv')
profile_df.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [3]:
# Count NaN values for each column
nan_counts = profile_df.isna().sum()

# Show only columns that have NaN values (if any)
columns_with_nans = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(columns_with_nans)

# For columns with NaN values, show unique Category-Topic combinations
if len(columns_with_nans) > 0:
    for col in columns_with_nans.index:
        print(f"\nUnique Category-Topic combinations where {col} is NaN:")
        unique_combinations = profile_df[profile_df[col].isna()][['Category', 'Topic']].drop_duplicates()
        print(unique_combinations)

Columns with NaN values:
City of Toronto                  7
Agincourt North                 56
Agincourt South-Malvern West    56
Alderwood                       56
Annex                           56
                                ..
Wychwood                        56
Yonge-Eglinton                  56
Yonge-St.Clair                  56
York University Heights         56
Yorkdale-Glen Park              56
Length: 141, dtype: int64

Unique Category-Topic combinations where City of Toronto is NaN:
                       Category                      Topic
0     Neighbourhood Information  Neighbourhood Information
1960            Journey to work      Commuting destination

Unique Category-Topic combinations where Agincourt North is NaN:
             Category                                    Topic
946            Income            Income of individuals in 2015
1014           Income             Income of households in 2015
1076           Income      Income of economic families in 2015
168

In [4]:
selected_row = {
  "Income of households in 2015": ["Average after-tax income of households in 2015 ($)"],
  "Mother tongue": [],
  "Age characteristics": [],
  "Immigrants by selected place of birth": [],
  "Recent immigrants by selected place of birth": [],
  "Population and dwellings": ["Land area in square kilometres"]
}

In [5]:
# stripe profile_df Characteristic and Topic
profile_df['Characteristic'] = profile_df['Characteristic'].str.strip()
profile_df['Topic'] = profile_df['Topic'].str.strip()

In [6]:
# Create a mask for filtering based on selected_row criteria
masks = []

for topic, characteristics in selected_row.items():
    if topic == "Mother tongue":
        # profile_df['Characteristic'] should not start from space
        topic_mask = (profile_df['Topic'] == topic) & ~(profile_df['Characteristic'].str.endswith('n.i.e.') | profile_df['Characteristic'].str.endswith('n.o.s.'))
    elif characteristics:  # If the list is not empty
        # Filter rows where Topic matches and Characteristic is in the list
        topic_mask = (profile_df['Topic'] == topic) & (profile_df['Characteristic'].isin(characteristics))
    else:
        # If list is empty, just match the Topic
        topic_mask = (profile_df['Topic'] == topic)
    masks.append(topic_mask)

# Combine all masks with OR operation
final_mask = pd.concat(masks, axis=0).groupby(level=0).any()

# Apply the mask to filter the DataFrame
filtered_profile_df = profile_df[final_mask]

filtered_profile_df.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
8,9,Population,Population and dwellings,Census Profile 98-316-X2016001,Land area in square kilometres,630.2,7.41,7.83,4.95,2.81,...,2.91,5.53,12.31,1.6,1.17,1.68,1.65,1.17,13.23,6.04
9,10,Population,Age characteristics,Census Profile 98-316-X2016001,Children (0-14 years),"398,135","3,840","3,075","1,760","2,360",...,"1,785","3,555","9,625","2,325","1,165","1,860","1,800","1,210","4,045","1,960"
10,11,Population,Age characteristics,Census Profile 98-316-X2016001,Youth (15-24 years),"340,270","3,705","3,360","1,235","3,750",...,"2,230","2,625","7,660","1,035",675,"1,320","1,225",920,"4,750","1,870"
11,12,Population,Age characteristics,Census Profile 98-316-X2016001,Working Age (25-54 years),"1,229,555","11,305","9,965","5,220","15,040",...,"7,480","8,140","21,945","6,165","3,790","6,420","5,860","5,960","12,290","5,860"
12,13,Population,Age characteristics,Census Profile 98-316-X2016001,Pre-retirement (55-64 years),"336,670","4,230","3,265","1,825","3,480",...,"2,070","2,905","6,245","1,625","1,150","1,595","1,325","1,540","2,965","1,810"


In [7]:
# 获取需要处理的列（从第6列开始）
value_columns = filtered_profile_df.columns[5:]

# 定义函数来转换字符串为数值
def convert_to_numeric(x):
    if pd.isna(x):  # 处理空值
        return x
    # 移除千位分隔符
    if isinstance(x, str):
        x = x.replace(',', '')
    # 转换为float
    try:
        return float(x)
    except:
        return x

# 应用转换到所有数值列
for col in value_columns:
    filtered_profile_df[col] = filtered_profile_df[col].apply(convert_to_numeric)

filtered_profile_df.head()

/var/folders/jw/8cqtqgln6l967hbckcxktsvr0000gn/T/ipykernel_88401/459504010.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_profile_df[col] = filtered_profile_df[col].apply(convert_to_numeric)


,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
8,9,Population,Population and dwellings,Census Profile 98-316-X2016001,Land area in square kilometres,630.2,7.41,7.83,4.95,2.81,...,2.91,5.53,12.31,1.6,1.17,1.68,1.65,1.17,13.23,6.04
9,10,Population,Age characteristics,Census Profile 98-316-X2016001,Children (0-14 years),398135.0,3840.00,3075.00,1760.00,2360.00,...,1785.00,3555.00,9625.00,2325.0,1165.00,1860.00,1800.00,1210.00,4045.00,1960.00
10,11,Population,Age characteristics,Census Profile 98-316-X2016001,Youth (15-24 years),340270.0,3705.00,3360.00,1235.00,3750.00,...,2230.00,2625.00,7660.00,1035.0,675.00,1320.00,1225.00,920.00,4750.00,1870.00
11,12,Population,Age characteristics,Census Profile 98-316-X2016001,Working Age (25-54 years),1229555.0,11305.00,9965.00,5220.00,15040.00,...,7480.00,8140.00,21945.00,6165.0,3790.00,6420.00,5860.00,5960.00,12290.00,5860.00
12,13,Population,Age characteristics,Census Profile 98-316-X2016001,Pre-retirement (55-64 years),336670.0,4230.00,3265.00,1825.00,3480.00,...,2070.00,2905.00,6245.00,1625.0,1150.00,1595.00,1325.00,1540.00,2965.00,1810.00


In [36]:
language_df = filtered_profile_df[filtered_profile_df['Topic'] == 'Mother tongue']
# unique Characteristic
original_language_characteristics = language_df['Characteristic'].unique()

filtered_language_characteristics = language_df[~language_df['Characteristic'].str.contains('languages|responses|Mother| and |Serbo-Croatian', case=True)]['Characteristic'].unique()

# 找出被过滤掉的语言特征
filtered_out_characteristics = set(original_language_characteristics) - set(filtered_language_characteristics)

# 显示被过滤掉的特征
print("被过滤掉的语言特征:")
for characteristic in sorted(filtered_out_characteristics):
    print(f"- {characteristic}")

# 检查是否有包含 'Mandarin' 的特征被过滤掉
mandarin_filtered_out = [char for char in filtered_out_characteristics if 'Mandarin' in char]
if mandarin_filtered_out:
    print("\n包含 'Mandarin' 的被过滤掉的特征:")
    for item in mandarin_filtered_out:
        print(f"- {item}")

被过滤掉的语言特征:
- Aboriginal languages
- Afro-Asiatic languages
- Algonquian languages
- Athabaskan languages
- Austro-Asiatic languages
- Austro-Asiatic languages, n.i.e
- Austronesian languages
- Baltic languages
- Balto-Slavic languages
- Berber languages
- Celtic languages
- Chinese languages
- Cree-Montagnais languages
- Creole languages
- Cushitic languages
- Dravidian languages
- Eastern Algonquian languages
- English and French
- English and non-official language
- English, French and non-official language
- French and non-official language
- Germanic languages
- Hmong-Mien languages
- Indo-Aryan languages
- Indo-European languages
- Indo-Iranian languages
- Inuit languages
- Iranian languages
- Iroquoian languages
- Italic (Romance) languages
- Karenic languages
- Kartvelian languages
- Mongolic languages
- Mother tongue for the total population excluding institutional residents
- Multiple responses
- Niger-Congo languages
- Nilo-Saharan languages
- Non-Aboriginal languages
- Non-o

In [37]:
language_df = language_df[~language_df['Characteristic'].str.contains('languages|responses|Mother| and |Serbo-Croatian', case=True)]
language_df['Characteristic'] = language_df['Characteristic'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.strip()
language_df

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
142,143,Language,Mother tongue,Census Profile 98-316-X2016001,English,1375905.0,7070.0,7080.0,7360.0,20645.0,...,5785.0,11470.0,22645.0,9515.0,5405.0,8445.0,8485.0,9230.0,11385.0,5900.0
143,144,Language,Mother tongue,Census Profile 98-316-X2016001,French,35440.0,120.0,140.0,105.0,705.0,...,140.0,265.0,400.0,325.0,120.0,230.0,240.0,260.0,195.0,100.0
147,148,Language,Mother tongue,Census Profile 98-316-X2016001,Blackfoot,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,155,Language,Mother tongue,Census Profile 98-316-X2016001,Babine,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155,156,Language,Mother tongue,Census Profile 98-316-X2016001,Beaver,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,382,Language,Mother tongue,Census Profile 98-316-X2016001,Uyghur,240.0,5.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
382,383,Language,Mother tongue,Census Profile 98-316-X2016001,Uzbek,440.0,0.0,0.0,0.0,0.0,...,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
385,386,Language,Mother tongue,Census Profile 98-316-X2016001,Estonian,1945.0,5.0,5.0,10.0,20.0,...,10.0,5.0,10.0,15.0,5.0,5.0,25.0,15.0,0.0,5.0
386,387,Language,Mother tongue,Census Profile 98-316-X2016001,Finnish,1315.0,0.0,10.0,5.0,5.0,...,10.0,10.0,10.0,10.0,10.0,5.0,15.0,15.0,5.0,0.0


In [38]:
neighbourhoods = filtered_profile_df.columns[5:]
base_cols = ['_id', 'Category', 'Topic', 'Data Source', 'Characteristic']

def revert_table_direction(df):
    melted_df = pd.melt(
        df,
        id_vars=base_cols,
        value_vars=neighbourhoods,
        var_name='neighbourhood',
        value_name='value'
    )

    pivoted_df = melted_df.pivot(
        index=['neighbourhood'],
        columns='Characteristic',
        values='value'
    ).reset_index()

    pivoted_df.columns.name = None

    return pivoted_df

In [39]:
filtered_language_df = revert_table_direction(language_df)
filtered_language_df.to_csv('../static/data/processed/filtered_languages.csv', index=False)

In [ ]:
pd.DataFrame(filtered_profile_df.loc[filtered_profile_df['Topic'] == 'Mother tongue', "Characteristic"].unique()).to_csv('../static/data/processed/language_characteristics.csv', index=False)

In [30]:
pd.DataFrame(language_df.loc[language_df['Topic'] == 'Mother tongue', "Characteristic"].unique()).to_csv('../static/data/processed/language_characteristics.csv', index=False)